In [1]:
import numpy as np
import scipy.sparse as sp
import pandas as pd
import networkx as nx

import sys
sys.path.append('..')
import src

Here we consider an idea social network that the whole community is fully connected, i.e., each household is connected to every other household with strong tie.

In [2]:
PARAMS = {
    'community': 'southpark',
    'experiment_name': 'ideal_social_net',
    # resource types should be the same as the column names in the resource_preference_pmf and resource_inventory_pmf
    'resource_types': ['water', 'food', 'meds', 'transp', 'comm',
                       'faid', 'warm', 'sani', 'power', 'shelter'], 
    'tau': 5, # isolation days
    'num_scenarios': 100,
    'n': 0.82661451, # degree distribution parameter
    'p': 0.06686762, # degree distribution parameter
    'social_tie': {
        'types': ['strong', 'weak'],
        'pmf':  [1, 0]
    },
    'distance_decay_alpha': -1.43, # distance decay parameter
    'distances_mat': np.load('../temp/southpark_distance_matrix.npy'), # distance matrix
    'sharing_preference_pmf': pd.read_csv('../temp/southpark_share_pref_PMF.csv'), # sharing preference matrix
    'sharing_priority': [3, 2, 1], # sharing priorities to [strong tie, weak tie, stranger]
    'resource_inventory_pmf': pd.read_csv('../temp/southpark_resource_distr_PMF.csv') # resource inventory distribution
}

In [3]:
def p2p_single_resource_sharing(resource:str, 
                                tau:float, # isolation days
                                n:float, # negative binomial distribution parameter
                                p:float, # negative binomial distribution parameter
                                distance_decay_alpha:float, # distance decay parameter
                                distance_mat:np.matrix, # distance matrix
                                sharing_preference_PMF:pd.Series, 
                                resource_inventory_PMF:pd.Series,
                                social_tie_pmf:list,
                                social_tie_types:list = ['strong', 'weak'],
                                sharing_priority:list = [3, 2, 1],
                                seed:int=0):
    """Step 1: Community-based social network construction"""
    # create a community
    community = src.Community()
    num_nodes = distance_mat.shape[0]
    community.add_nodes_from(range(num_nodes))

    # generate a fully connected social network
    community.add_edges_from(nx.complete_graph(num_nodes).edges)
    community.split_social_ties(types=social_tie_types, 
                                pmf=social_tie_pmf, 
                                seed=seed+2)

    """Step 2: Community-based resource-sharing network construction"""
    # generate sharing preferences
    community.generate_sharing_preference(recipient=[0, 1, 2, 3], # [nobody, strong tie only, strong and weak ties only, everybody]
                                          pmf=sharing_preference_PMF, 
                                          name=resource+'_share_preference',
                                          seed=seed+3)

    """Step 3: Community-based resource inventory"""
    community.generate_resource_inventory(support=np.arange(0, len(resource_inventory_PMF)), 
                                          pmf=resource_inventory_PMF, 
                                          name=resource, 
                                          seed=seed+4)

    # create sharing network
    share_network = community.get_share_network(sharing_preference=resource+'_share_preference', 
                                                priority=sharing_priority)

    """Step 4: Community-based P2P resource-sharing model"""
    priority_matrix = community.get_share_priority_matrix(sharing_preference=resource+'_share_preference', 
                                                          priority=sharing_priority)
    resource_inventory = [community.nodes[n][resource] for n in community.nodes]
    model = src.ResourceSharingModel(wts_matrix=priority_matrix, 
                                     inventory=resource_inventory, 
                                     isolation_days=tau)
    model.solve()

    return community, share_network, model

In [4]:
# main script for running the status quo model
for sce in range(PARAMS['num_scenarios']):
    
    # directory name for saving the results
    dir = PARAMS['community'] + '/' + PARAMS['experiment_name'] + '/tau_' + str(PARAMS['tau']) # directory name
    # Create a dataframe to store the results
    inventory_df = pd.DataFrame(columns=PARAMS['resource_types'])
    resource_df = pd.DataFrame(columns=PARAMS['resource_types'])
    
    # run the model for each resource type
    for rsc in PARAMS['resource_types']:
        community, share_network, model = p2p_single_resource_sharing(resource=rsc,
                                                                    tau=PARAMS['tau'],
                                                                    n=PARAMS['n'],
                                                                    p=PARAMS['p'],
                                                                    social_tie_types = PARAMS['social_tie']['types'],
                                                                    social_tie_pmf = PARAMS['social_tie']['pmf'],
                                                                    distance_decay_alpha = PARAMS['distance_decay_alpha'],
                                                                    distance_mat = PARAMS['distances_mat'],
                                                                    sharing_preference_PMF = PARAMS['sharing_preference_pmf'][rsc],
                                                                    sharing_priority = PARAMS['sharing_priority'],
                                                                    resource_inventory_PMF = PARAMS['resource_inventory_pmf'][rsc],
                                                                    seed=sce)
        inventory_df[rsc] = model.inventory.flatten()
        resource_df[rsc] = model.resource.flatten()

    # save the resource inventory vector in csv file
    inventory_df.to_csv(dir + '/q_vec_sce_{}.csv'.format(sce), index=False)
    # save the final resource vector in csv file
    resource_df.to_csv(dir + '/h_vec_sce_{}.csv'.format(sce), index=False)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-31
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10700 CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2758 rows, 1901641 columns and 3803282 nonzeros
Model fingerprint: 0xda292d4b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 1518 rows and 1552585 columns
Presolve time: 0.72s
Presolved: 1240 rows, 349056 columns, 698112 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2740000e+03   8.670876e-01   0.000000e+00      1s
       9    2.2740000e+03   0.000000e+00   0.000000e+00      1s

Use crossover to convert LP symmetric solution to basic solution...
Crossover log...

       0 DPushes remainin